In [2]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
movies=pd.read_csv('movies.csv')
movies.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [4]:
movies

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

###Printing the first five columns

In [5]:
movies.head()


,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [6]:
movies.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [7]:
movies.shape

(4803, 24)

#We need only few columns from this for our recommendation. we are taking those columns and removing the null values from that columns with a null string

In [8]:
needed=('genres','keywords','tagline','cast','director')
print(needed)

('genres', 'keywords', 'tagline', 'cast', 'director')


In [9]:
# Replacing the null values in the required dataset with a null string
for a in needed:
    movies[a]=movies[a].fillna('')

In [10]:
movies.isnull().sum()

index                      0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                    0
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
director                   0
dtype: int64

#we are creating a dataset called recommender that has all the reqired columns combined together

In [11]:
recommender=movies['genres']+movies['keywords']+movies['tagline']+movies['cast']+movies['director']
print(recommender)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object


In [12]:
print(recommender[0])

Action Adventure Fantasy Science Fictionculture clash future space war space colony societyEnter the World of Pandora.Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle RodriguezJames Cameron


In [13]:
#converting the text data into numeric values since the machine can understand only numerical values.
#conversion is done using Tfidf Vectorizer imported from sklearn



In [14]:
converter=TfidfVectorizer() 

In [15]:
features=converter.fit_transform(recommender)

In [16]:
print(features)

  (0, 3583)	0.16941894714909375
  (0, 20729)	0.2738578969459924
  (0, 16614)	0.15696073877453268
  (0, 14550)	0.2236809363648068
  (0, 23267)	0.16128139780622516
  (0, 26547)	0.19638671147741732
  (0, 22464)	0.20207089568711745
  (0, 21487)	0.21550128478931546
  (0, 27540)	0.19771357974524176
  (0, 27182)	0.23480088356130552
  (0, 21521)	0.1562356665943368
  (0, 18558)	0.2612168335104853
  (0, 18164)	0.08690831799482265
  (0, 27161)	0.1261988657937151
  (0, 24197)	0.07518543993419265
  (0, 22778)	0.2738578969459924
  (0, 4676)	0.2452909770424961
  (0, 26334)	0.13059723071916296
  (0, 22916)	0.33668756406929184
  (0, 10193)	0.16532432420812987
  (0, 4456)	0.21799675215510664
  (0, 9304)	0.2738578969459924
  (0, 21836)	0.09966592997173944
  (0, 8936)	0.11806131645084653
  (0, 444)	0.09109355212252294
  :	:
  (4801, 5449)	0.31510541707008866
  (4801, 22170)	0.31510541707008866
  (4801, 27361)	0.31510541707008866
  (4801, 27513)	0.3005604008026175
  (4801, 8121)	0.2611505149930138
  (4801,

# Here we use a new function called as cosine_similarity function

In [17]:
similarity=cosine_similarity(features)
print(similarity)

[[1.         0.06865296 0.01492221 ... 0.         0.         0.        ]
 [0.06865296 1.         0.02799128 ... 0.01243107 0.         0.        ]
 [0.01492221 0.02799128 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01243107 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [18]:
print(similarity[0])

[1.         0.06865296 0.01492221 ... 0.         0.         0.        ]


In [19]:
print(similarity[0].shape)

(4803,)


In [22]:
print(similarity.shape)

(4803, 4803)


In [23]:
#getting the input from the user
user_movie=input('Enter your favourite movie name:')

Enter your favourite movie name:iron man


In [24]:
title_list=movies['title'].tolist()
print(title_list)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [25]:
related_movies=difflib.get_close_matches(user_movie,title_list)
print(related_movies)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [26]:
close_match=related_movies[0]
print(close_match)

Iron Man


In [27]:
index=movies[movies.title==close_match]['index'].values[0]
print(index)

68


In [28]:
similarity_score=list(enumerate(similarity[index]))
print(similarity_score)

[(0, 0.015106149141252807), (1, 0.034008545917263946), (2, 0.005970688419991261), (3, 0.006346189192212358), (4, 0.01480427477891974), (5, 0.005949882865469456), (6, 0.051130903895656), (7, 0.14475303569590786), (8, 0.0), (9, 0.035427843936016785), (10, 0.01745293694932597), (11, 0.005374696360944075), (12, 0.0), (13, 0.005611099160776941), (14, 0.04517825188687174), (15, 0.0), (16, 0.18564323259620552), (17, 0.00569896217764925), (18, 0.014698962737314466), (19, 0.005079920020055092), (20, 0.06806755551861558), (21, 0.005173556735249874), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.1603230693292908), (27, 0.01366486696637728), (28, 0.05629494939086527), (29, 0.0061362061886234085), (30, 0.07040313185709823), (31, 0.22094451191029774), (32, 0.019972034195043942), (33, 0.12555697271128102), (34, 0.0), (35, 0.03187039186963796), (36, 0.02928883361682064), (37, 0.0), (38, 0.1020035754254405), (39, 0.07465498761376428), (40, 0.0), (41, 0.01434399611538337), (42, 0.0), (43, 0.0300002

In [29]:
len(similarity_score)

4803

In [30]:
sorted_list=sorted(similarity_score,key=lambda x:x[1],reverse=True)
print(sorted_list)

[(68, 1.0), (79, 0.23663084372818138), (31, 0.22094451191029774), (16, 0.18564323259620552), (26, 0.1603230693292908), (4401, 0.15455439588528466), (7, 0.14475303569590786), (33, 0.12555697271128102), (46, 0.12091625223248503), (511, 0.11447511448088141), (203, 0.11302588802563136), (3623, 0.11195822443347257), (353, 0.11142595449707034), (174, 0.10864468060155888), (64, 0.10372950313359512), (38, 0.1020035754254405), (85, 0.10145942826705964), (1210, 0.10053490492219067), (101, 0.10030515395125751), (2235, 0.09948877294015109), (954, 0.09853929898816832), (788, 0.09321494772760432), (3166, 0.09246756760167425), (1406, 0.09228343737779446), (882, 0.0908411980573617), (2186, 0.08751927538174428), (2063, 0.08528803891611837), (182, 0.08459938579039519), (1282, 0.0841272768737229), (3443, 0.08399261269675581), (307, 0.08396840450529347), (356, 0.08013663583472504), (205, 0.07913407395905812), (2066, 0.07786448224504089), (4542, 0.07735936472905562), (783, 0.07722722111413247), (2411, 0.07

In [31]:
print("MOVIES SUGGESTED FOR YOU!\n")
i=1
for x in sorted_list:
  index_number=x[0]
  suggestions=movies[movies.index==index_number]['title'].values[0]
  print(suggestions)

MOVIES SUGGESTED FOR YOU!

Iron Man
Iron Man 2
Iron Man 3
The Avengers
Captain America: Civil War
The Helix... Loaded
Avengers: Age of Ultron
X-Men: The Last Stand
X-Men: Days of Future Past
X-Men
X2
Made
Tropic Thunder
The Incredible Hulk
X-Men: Apocalypse
The Amazing Spider-Man 2
Captain America: The Winter Soldier
Gothika
X-Men: First Class
A Scanner Darkly
The Judge
Deadpool
The Best Man
The Nativity Story
Lucky You
The Kite Runner
Arlington Road
Ant-Man
The Hunting Party
Mallrats
The Expendables 3
Sherlock Holmes
Sherlock Holmes: A Game of Shadows
Firefox
Monsters
Mortdecai
The Best Man Holiday
Super
Kiss Kiss Bang Bang
Fur: An Imaginary Portrait of Diane Arbus
Fortress
Crazy Heart
TRON: Legacy
The Men Who Stare at Goats
Godsend
Se7en
Zathura: A Space Adventure
K-PAX
Winnie Mandela
In Dreams
Charlie Bartlett
The Man
Dead Man Down
Zodiac
Spider-Man 2
Happily N'Ever After
The Singing Detective
The Shaggy Dog
The Giver
Awake
The Open Road
The Salon
Angel Eyes
The Amazing Spider-Man
K

In [32]:
print("MOVIES SUGGESTED FOR YOU!\n")
i=1
for x in sorted_list:
  index_number=x[0]
  suggestions=movies[movies.index==index_number]['title'].values[0]
  if (i<16):
    print(i,suggestions)
    i+=1

MOVIES SUGGESTED FOR YOU!

1 Iron Man
2 Iron Man 2
3 Iron Man 3
4 The Avengers
5 Captain America: Civil War
6 The Helix... Loaded
7 Avengers: Age of Ultron
8 X-Men: The Last Stand
9 X-Men: Days of Future Past
10 X-Men
11 X2
12 Made
13 Tropic Thunder
14 The Incredible Hulk
15 X-Men: Apocalypse


# MOVIE RECOMMENDATION SYSTEM

In [34]:
user_input= input('Enter the name of your favourite movie:')

title_list=movies['title'].tolist()

related_movies=difflib.get_close_matches(user_input,title_list)

close_match=related_movies[0]

index=movies[movies.title==close_match]['index'].values[0]

similarity_score=list(enumerate(similarity[index]))

sorted_list=sorted(similarity_score,key=lambda x:x[1],reverse=True)

print("MOVIES SUGGESTED FOR YOU!\n")
i=1
for x in sorted_list:
  index_number=x[0]
  suggestions=movies[movies.index==index_number]['title'].values[0]
  if (i<16):
    print(i,suggestions)
    i+=1

Enter the name of your favourite movie:spectre
MOVIES SUGGESTED FOR YOU!

1 Spectre
2 Skyfall
3 Mission: Impossible - Ghost Protocol
4 Carnage
5 Quantum of Solace
6 Water for Elephants
7 The Green Hornet
8 The Legend of Tarzan
9 Django Unchained
10 Inglourious Basterds
11 Heavenly Creatures
12 Die Another Day
13 Big Eyes
14 Coriolanus
15 Spawn
